In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

import re
import pandas as pd
import time
import requests

In [2]:
brand=[]
product=[]
score_lst=[]
review_lst=[]
for line in range(1,26):    
    for i in range(4):
        url='https://www.oliveyoung.co.kr/store/main/getBestList.do'
        driver=webdriver.Chrome()
        driver.get(url)
        act=ActionChains(driver)
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        
        # 상세페이지 진입
        driver.find_elements(By.CSS_SELECTOR,'#Container > div.best-area > div.TabsConts.on > ul:nth-child('+str(line)+') > li')[i].click()
        
        # 브랜드, 상품 이름
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        brand.append(soup.select('div.prd_info >p >a')[0].text)
        product.append(soup.select('div.prd_info >p.prd_name')[0].text)
        review=[]
        score=[]

        try:
            # 리뷰창 이동
            driver.find_elements(By.CSS_SELECTOR,'#reviewInfo')[0].click()
            time.sleep(2)

            # 최신순으로 보기
            driver.find_elements(By.CSS_SELECTOR,'#gdasSort > li:nth-child(3)')[0].click()
            time.sleep(1)

        # 리뷰 크롤링
            for idx in range(10):
                html=driver.page_source
                soup=BeautifulSoup(html,'html.parser')
                time.sleep(1)
                score.append(soup.select('span.review_point')[idx+1].text)
                review.append(soup.select('div.txt_inner')[idx].text)

            # 리뷰창 다음 페이지 이동 및 리뷰 크롤링
            for page in range(2,11):
                driver.find_elements(By.CSS_SELECTOR,'#gdasContentsArea > div > div.pageing > a:nth-child('+str(page)+')')[0].click()
                for idx in range(10):
                    html=driver.page_source
                    soup=BeautifulSoup(html,'html.parser')
                    time.sleep(1)
                    score.append(soup.select('span.review_point')[idx+1].text)
                    review.append(soup.select('div.txt_inner')[idx].text)
        except:
            score.append('')
            review.append('')
            continue
        
        review_lst.append(review)
        score_lst.append(score)

        # 진행사항 확인
        if len(brand)%10==0:
            print(len(brand))
            
        # 창 닫고 새창열기
        driver.close()

10
20
30
40
50
60
70
90
100


In [4]:
print(len(brand))
print(len(product))
print(len(score_lst))
print(len(review_lst))

100
100
92
92


In [ ]:
df=pd.DataFrame({'brand':brand,'product':product,'score':score_lst,'review':review_lst})
df

In [134]:
df.to_csv('./data/olive0_review.csv',encoding='utf-8-sig',index=False)